In [15]:
import torch
import torch.nn as nn
import numpy as np
import math
from torch.utils.data import DataLoader
from ArgoverseDataset import ArgoverseForecastDataset
from vectornet import VectorNet
import warnings
warnings.filterwarnings('ignore')
import logging
import os

# ADE 和 FDE 计算函数
def get_ade(forecasted_trajectory: np.ndarray, gt_trajectory: np.ndarray) -> float:
    pred_len = forecasted_trajectory.shape[0]
    ade = float(
        sum(
            math.sqrt(
                (forecasted_trajectory[i, 0] - gt_trajectory[i, 0]) ** 2
                + (forecasted_trajectory[i, 1] - gt_trajectory[i, 1]) ** 2
            )
            for i in range(pred_len)
        )
        / pred_len
    )
    return ade

def get_fde(forecasted_trajectory: np.ndarray, gt_trajectory: np.ndarray) -> float:
    fde = float(
        math.sqrt(
            (forecasted_trajectory[-1, 0] - gt_trajectory[-1, 0]) ** 2
            + (forecasted_trajectory[-1, 1] - gt_trajectory[-1, 1]) ** 2
        )
    )
    return fde


In [16]:
def init_logger(cfg):
    logger = logging.getLogger(__name__)
    logger.setLevel(level=logging.DEBUG)
    log_file = cfg['log_file']
    handler = logging.FileHandler(log_file, mode='w')
    handler.setLevel(logging.DEBUG)

    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    logger.addHandler(handler)
    logger.addHandler(console)
    return logger


In [17]:
def load_model(cfg, model_path):
    device = cfg['device']
    model = VectorNet(traj_features=15, map_features=8, cfg=cfg)
    # Load state_dict from checkpoint
    state_dict = torch.load(model_path)
    
    # Handle the case where model was wrapped in DataParallel during training
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if k.startswith("module."):
            new_state_dict[k[7:]] = v  # remove "module." prefix
        else:
            new_state_dict[k] = v
    
    model.load_state_dict(new_state_dict)
    model.to(device)
    model.eval()
    return model

def predict(model, data_loader, cfg):
    device = cfg['device']
    predictions = []
    with torch.no_grad():
        for traj_batch, map_batch in data_loader:
            traj_batch = traj_batch.to(device=device, dtype=torch.float)
            result, _ = model(traj_batch, map_batch)
            predictions.append(result)
    return predictions

In [18]:
def evaluate_model(model, cfg):
    test_dataset = ArgoverseForecastDataset(cfg)
    test_loader = DataLoader(dataset=test_dataset, batch_size=cfg['batch_size'], shuffle=False, num_workers=0)
    
    predictions = predict(model, test_loader, cfg)
    
    total_ade = 0
    total_fde = 0
    count = 0
    
    for i, (traj_batch, _) in enumerate(test_loader):
        gt_trajectory = traj_batch[:, cfg['last_observe']:, 2:4].cpu().numpy()
        pred_trajectory = predictions[i]['0'].cpu().numpy()
        print(pred_trajectory)
        print(len(gt_trajectory))
        for j in range(len(gt_trajectory)):
            ade = get_ade(pred_trajectory[j], gt_trajectory[j])
            fde = get_fde(pred_trajectory[j], gt_trajectory[j])
            total_ade += ade
            total_fde += fde
            count += 1
    
    avg_ade = total_ade / count
    avg_fde = total_fde / count
    
    print(f"Average ADE: {avg_ade}")
    print(f"Average FDE: {avg_fde}")

In [19]:
USE_GPU = True
RUN_PARALLEL = True
device_ids = [0, 1]
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    if torch.cuda.device_count() <= 1:
        RUN_PARALLEL = False
        pass
else:
    device = torch.device('cpu')
    RUN_PARALLEL = False

learning_rate = 1e-3
learning_rate_decay = 0.3
cfg = dict(device=device, learning_rate=learning_rate, learning_rate_decay=learning_rate_decay,
            last_observe=20, epochs=3, print_every=100, save_every=1, batch_size=1, predict_step=29,
            data_locate="../argoverse-forecaster/data", save_path="./model_ckpt/",
            log_file="./log_eval.txt", tensorboard_path="runs/train_visualization")

print('RUN_PARALLEL = ' + str(RUN_PARALLEL))
print()

logger = init_logger(cfg)
model_path = cfg['save_path'] + "model_final.pth"
model = load_model(cfg, model_path)

evaluate_model(model, cfg)


RUN_PARALLEL = True

PIT map range :
[4890.21696825 2722.98842652]
[ 650.30176714 -203.72591182]
MIA map range :
[ 971.37104584 4210.89633146]
[-494.68405609  552.63750775]


Generating Vector Map: : 17526it [00:02, 8469.92it/s]                         


Generate Vector Map Successfully!
[[-5.8235973e-03  5.0490282e-02]
 [-9.4969980e-03  1.4169976e-01]
 [-4.4123214e-02  2.4500379e-01]
 [-5.4010846e-02  4.3678561e-01]
 [-5.7107206e-02  6.4537406e-01]
 [-7.8717686e-02  9.0424210e-01]
 [-8.1951782e-02  1.2053852e+00]
 [-1.0589730e-01  1.5638622e+00]
 [-1.2559189e-01  1.9375587e+00]
 [-1.5492673e-01  2.3518853e+00]
 [-1.7099653e-01  2.8465703e+00]
 [-1.8343438e-01  3.4052067e+00]
 [-2.1540819e-01  3.9850321e+00]
 [-2.2631733e-01  4.5803871e+00]
 [-2.5461608e-01  5.2373986e+00]
 [-2.8018633e-01  5.9333115e+00]
 [-3.2266027e-01  6.6893711e+00]
 [-3.4959626e-01  7.4814668e+00]
 [-3.9149979e-01  8.3155727e+00]
 [-4.3121168e-01  9.1743155e+00]
 [-4.3258676e-01  1.0115678e+01]
 [-4.8082805e-01  1.1113374e+01]
 [-5.1927632e-01  1.2117119e+01]
 [-5.7825303e-01  1.3162250e+01]
 [-6.2134606e-01  1.4249314e+01]
 [-6.8452972e-01  1.5384360e+01]
 [-7.4555475e-01  1.6531813e+01]
 [-8.0640757e-01  1.7730875e+01]
 [-8.8221079e-01  1.9006824e+01]]
1


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed